# Indhent tekster for hver url

In [14]:
import json

In [15]:
# Indlæs dictionary
with open("urls-depth-2.json", "r") as f:
    dic = json.load(f) 

In [16]:
CATEGORIES = list(dic.keys())
CATEGORIES

['Uddannelse',
 'Samfund',
 'Videnskab',
 'Natur',
 'Teknologi',
 'Kultur',
 'Historie',
 'Sundhed',
 'Geografi',
 'Økonomi',
 'Sport',
 'Religion',
 'Politik',
 'Erhvervsliv']

In [17]:
import re
import time
import numpy as np
from bs4 import BeautifulSoup
import requests

In [28]:
def get_plain_text(URL, max_tokens=200):
  # Try to request wikipedia text
  try:
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    s = soup.findAll('div',attrs={'id':"mw-content-text"})[0]
    content_text = s.findAll('div')[0]
  except:
    print(f"Failed for URL: {URL}. We try to obtain the wikitext again in 5 seconds...")
    try:
      time.sleep(5)
      response = requests.get(URL)
      soup = BeautifulSoup(response.text, 'html.parser')
      s = soup.findAll('div',attrs={'id':"mw-content-text"})[0]
      content_text = s.findAll('div')[0]
    except:
      print("Failed again - no text is collected.")
      return np.nan, np.nan

  # Get the text
  plain_text = ""
  for c in content_text:
      if not c.name in ["style", "div", "h1", "h2", "h3", "h4", "h5", "h6", "table", "dl"]:
          try:
            plain_text = plain_text + " " + c.text
          except:
            pass
  
  # Process the text
  plain_text = " ".join(plain_text.split())
  plain_text = re.sub("[\(\[].*?[\)\]]", "", plain_text).replace("  ", " ")
  plain_text = plain_text.replace("\'", "").replace("\"", "").replace("”", "").replace(" .", ".").replace(" ,", ",")
  
  # Get title
  try:
      title = soup.title.text.replace("- Wikipedia, den frie encyklopædi","").strip()
  except:
      title = "Title not detected"

  # Return text + title
  if max_tokens == 0:
    return " ".join(plain_text.split()), title
  else:
    return " ".join(plain_text.split()[:max_tokens]), title

# Test funktion
get_plain_text('https://da.wikipedia.org/wiki/Struktur')

('Struktur indre opbygning –. En struktur angår generelt den måde elementer er organiseret i forhold til hinanden og i forhold til den helhed de indgår i. Dette kan være fysisk, rumligt eller systematisk. Struktur kan have flere betydninger inden for forskellige fagområder og henvise til forskellige artikler: Struktur – et dansk forlag: Forlaget Struktur Struktur Struktur Struktur Struktur Struktur Struktur System Datastruktur Infrastruktur Overfladestruktur Organisationsstruktur Virksomhedsstruktur',
 'Struktur')

## Scrape Wikipedia artikler

In [19]:
%%time
text_list = []
label_num_list = []
label_list = []
title_list = []

for ix, cat in enumerate(CATEGORIES):
    for url in dic[cat]:
        [text, title] = get_plain_text(url)
        if len(text)>1:
            text_list.append(text)
            label_num_list.append(ix)
            label_list.append(cat)
            title_list.append(title)
    print(cat + f" {ix+1}/{len(CATEGORIES)}")



Uddannelse 1/14
Samfund 2/14
Videnskab 3/14
Natur 4/14
Teknologi 5/14
Kultur 6/14
Historie 7/14
Sundhed 8/14
Geografi 9/14
Økonomi 10/14
Sport 11/14
Religion 12/14
Politik 13/14
Erhvervsliv 14/14
CPU times: total: 18min 28s
Wall time: 1h 3min 37s


# Gem tekster i csv-fil

In [20]:
import pandas as pd

In [21]:
dict_df = {"text":text_list, "label_num": label_num_list, "label": label_list, "title_list":title_list}

In [22]:
df = pd.DataFrame.from_dict(dict_df)

In [23]:
df.head()

,text,label_num,label,title_list
0,"Uddannelse er et udtryk, der både benyttes om ...",0,Uddannelse,Uddannelse
1,30 er en dansk dokumentarfilm fra 2013 instrue...,0,Uddannelse,30 (dokumentarfilm)
2,"Accessit fra latin for han er kommet nær til, ...",0,Uddannelse,Accessit
3,"Almen dannelse er et begreb, der bruges om en ...",0,Uddannelse,Almendannelse
4,"En alumnus eller alumne er altid en person, me...",0,Uddannelse,Alumne


In [24]:
df.tail()

,text,label_num,label,title_list
13950,"En selvejende institution er et foretagende, d...",13,Erhvervsliv,Selvejende institution
13951,"En skibsreder er en person, der ejer et eller ...",13,Erhvervsliv,Skibsreder
13952,"En social virksomhed er en virksomhed, der er ...",13,Erhvervsliv,Social virksomhed
13953,En socialøkonomisk virksomhed er en virksomhed...,13,Erhvervsliv,Socialøkonomisk virksomhed
13954,"Et § 60-selskab er en virksomhed, der er opret...",13,Erhvervsliv,§ 60-selskab


In [25]:
df.to_csv("total_raw_wikidata.csv", index=False)